# Imports

In [29]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data loading

In [34]:
data_path = '/content/drive/MyDrive/ZIwG P/citations.csv'
df = pd.read_csv(data_path)
df.head()

,content,author,label
0,O ile my podnieśliśmy pensję minimalną w czasi...,Izabela Leszczyna,Fałsz
1,W rankingu praworządności World Justice Projec...,Hanna Gill-Piątek,Prawda
2,Na Uniwersytecie Warszawskim powstał taki rapo...,Dariusz Rosati,Prawda
3,Średnia emerytura w Polsce kobiet i mężczyzn r...,Małgorzata Kidawa-Błońska,Prawda
4,Proszę mi pokazać (…) jedną osobę skazaną prze...,Paweł Kukiz,Fałsz


# Text preprocessing

# Model nr 1

# Model nr 2

# Model nr 3

# Evaluation